# Web Search & Summarization Agent with Smolagent

In this notebook, we will build a natural language agent capable of:

- Searching the web based on user queries  
- Automatically extracting and summarizing content from the resulting pages  
- Returning structured answers, powered by a large language model (LLM)

We will use the [`smolagents`](https://github.com/smol-ai/smolagent) framework to connect tools like `DuckDuckGo` search and HTML scraping with an OpenAI model.


## Example use case

> "Summarize the Hugging Face blog post about open LLMs from 2023."

The agent will:
1. Search the web for that blog post
2. Find the best link
3. Read and clean the content of the page
4. Summarize

In [1]:
import os
from smolagents import CodeAgent
from smolagents import OpenAIServerModel
from dotenv import load_dotenv

load_dotenv(override=True)

/Users/raphael/workshop-smolagents-mcp/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Tool: `web_search(query: str)`

This tool enables the agent to **search the web using DuckDuckGo**.  
It returns the **top 3 relevant results**, each with a title and a clickable URL.

LLMs don’t have live access to the internet by default.  

Example queries the agent might use this tool for:
- `"latest news about open-source LLMs"`
- `"hugging face blog post on quantization"`
- `"datacraft website upcoming events"`

#### How it works

- The tool uses the [`ddgs`](https://pypi.org/project/ddgs/) library (DuckDuckGo Search).
- It performs a web search for the given query.
- It returns the first 3 results as a bullet list: `"- [title] ([url])"`

In [2]:
from workshop_smolagents_mcp.tools import web_search

The `query_events_db` tool is defined in the [tools.py](./tools.py) file.

### Tool: `summarize_url(url: str)`

This tool allows the agent to **extract and summarize the main textual content** from a public web page.

It plays a key role in helping the agent “read the web” by turning a full HTML page into a clean, readable excerpt — ready to be summarized by the LLM.

After finding a link with `web_search(...)`, the agent needs to access and understand what’s on that page.  

This tool:
- Downloads the page (`requests`)
- Parses it (`BeautifulSoup`)
- Removes all unnecessary tags (e.g. `<script>`, `<style>`, `<noscript>`)
- Extracts **only the visible text**
- Returns the **first 60 lines** as a preview, making it manageable for summarization



- **Input**: a valid URL (starting with `http://` or `https://`)
- **Output**: a cleaned block of raw text, ready to be passed to the model

If the page is unreachable, the tool returns a fallback error message.

The `summarize_url` tool is defined in the [tools.py](./tools.py) file.

In [3]:
from workshop_smolagents_mcp.tools import summarize_url

In [4]:
model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"]
)

### Creating an Agent with Multiple Tools

Now that we’ve defined two tools — `web_search` and `summarize_url` — we can combine them into a single agent.

By passing **both tools** to the `CodeAgent`, the model gains the ability to:

1. **Search the web** for relevant information (via `web_search`)
2. **Read and extract the content** of a webpage (via `summarize_url`)


The `CodeAgent` uses the language model to reason about the user’s question and **decide when and how to call each tool**.  
This means you don’t need to manually control the flow — the agent will:

- Interpret the user's intent
- Call one or both tools as needed
- Return a natural language answer

 For example, if you ask:
> *"Summarize the latest blog post from Hugging Face about open LLMs"*  
 
 The agent might:
1. Use `web_search` to find the link
2. Use `summarize_url` on that link
3. Return a final summary

In [5]:
agent = CodeAgent(
    tools=[web_search, summarize_url],
    model=model,
)

In [6]:
question = "What is datacraft paris?"
agent.run(question)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is datacraft paris?                                                                                        │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pages = web_search(query="datacraft paris")                                                                      
  print(pages)                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
- datacraft Club pour data scientists - datacraft (https://datacraft.paris/)
- datacraft : photos, vidéos, recrutement (https://www.welcometothejungle.com/fr/companies/datacraft)
- datacraft - paris ( datacraft - paris ) · GitHub (https://github.com/datacraft-paris)

Out: None

[Step 1: Duration 1.96 seconds| Input tokens: 1,982 | Output tokens: 75]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("datacraft Paris is a learning and coworking club for data scientists and data engineers,           
  fostering collaboration, innovation, and implementation of data and AI projects in Paris.")                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: datacraft Paris is a learning and coworking club for data scientists and data engineers, fostering 
collaboration, innovation, and implementation of data and AI projects in Paris.

[Step 3: Duration 2.41 seconds| Input tokens: 7,118 | Output tokens: 309]

'datacraft Paris is a learning and coworking club for data scientists and data engineers, fostering collaboration, innovation, and implementation of data and AI projects in Paris.'